[프롬프트 엔지니어링, LLM(ChatGPT) 기반의 AI 서비스 개발](https://fastcampus.co.kr/data_red_golbin) 강의에서 사용하는 예제입니다.

In [1]:
# !pip install openaiW

In [15]:
%pwd

'C:\\Users\\runia\\Desktop\\gpt_수능영어_파인튜닝'

# 1. 데이터 준비

In [1]:
from openai import OpenAI
import pandas as pd
import numpy as np
import ast
import json
from tqdm import tqdm_notebook
import re

import warnings
warnings.filterwarnings(action='ignore')

from dotenv import load_dotenv
load_dotenv()

import os
api_key = os.environ.get('OPENAI_API_KEY')

client = OpenAI(api_key = api_key)

## (1) jsonl 데이터 경우

In [2]:
# jsonl 데이터 로드
data_list = []
with open("data/data4_10type.jsonl", "r", encoding='utf8') as f:
    for line in f:
        data_list.append( json.loads(line) )        

df = pd.DataFrame(data_list)

# 질문+지문+옵션 합치기
all_question_list = []
for i in range(len(df)):
    
    all_question = df['question'][i] + '\n' + df['article'][i] + '\n' + df['options'][i] + '\n'
    print(all_question)
    all_question_list.append(all_question)

# 답변 
answer_list = []

for i in range(len(df)):
    answer = df['solution'][i] + '\n'
    print(answer)
    answer_list.append(answer)

df_que_ans = pd.DataFrame({"full_question" : all_question_list, "answer": answer_list})

다음 글의 목적으로 가장 적절한 것은?
To whom it may concern,
I am writing to inform you of an ongoing noise issue that 
I am experiencing. My apartment faces the basketball 
courts of the community center. While I fully support the 
community center’s services, I am constantly being disrupted 
by individuals playing basketball late at night. Many
nights, I struggle to fall asleep because I can hear people 
bouncing balls and shouting on the basketball courts well 
after 11 p.m.. Could you restrict the time the basketball 
court is open to before 9 p.m.? I’m sure I’m not the only
person in the neighborhood that is affected by this noise 
issue. I appreciate your assistance.
Sincerely, 
Ian Baldwin
① 체육관의 바닥 교체 공사를 요구하려고 
② 농구 코트의 운영 시간 제한을 요청하려고
③ 문화 센터 시설의 대관 날짜를 변경하려고
④ 건강 증진 프로그램 신청 방법을 문의하려고
⑤ 지역 내 체육 시설의 증설 가능 여부를 확인하려고

다음 글의 목적으로 가장 적절한 것은?
To whom it may concern,
I would like to draw your attention to a problem that 
frequently occurs with the No. 35 buses. There is a bus 
stop about halfway a

# (2) csv 데이터 경우

In [3]:
df_que_ans = pd.read_csv("data/ft_data5_36~39.csv")
df_que_ans

,full_question,testNumber,question,questionNumber,article,options,answer,questionType,solution,keyWords,난이도(예정),오답률(예정),Unnamed: 12
0,Q. 주어진 context 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.\...,2_2312,Q. 주어진 context 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.,36,<context>\n The size of a species is not accid...,1. (A) -> (C) -> (B)\n2. (B) -> (A) -> (C)\n3....,2,NaN,"{'정답':2, 'reason': '주어진 context는 종의 크기가 환경과 상호...","""'size':'크기' | 'trend':'경향' | 'environment':'환경'""",NaN,NaN,NaN
1,Q. 주어진 글 다음에 이어질 글의 순서로 가장 적절한 것을 \n고르시오.\nIf ...,2_2309,Q. 주어진 글 다음에 이어질 글의 순서로 가장 적절한 것을 \n고르시오.,36,"If you drive down a busy street, you will find...",1. (A) -> (C) -> (B)\n2. (B) -> (A) -> (C)\n3....,5,NaN,"{'정답':5, 'reason': '주어진 글은 많은 경쟁 업체들이 나란히 위치하고...","""'competing':'경쟁하는' | 'advertising':'광고' | 'di...",NaN,NaN,NaN
2,Q. 주어진 글 다음에 이어질 글의 순서로 가장 적절한 \n것을 고르시오.\n\nW...,2_2306,Q. 주어진 글 다음에 이어질 글의 순서로 가장 적절한 \n것을 고르시오.,36,"When evaluating a policy, people tend to conce...",1. (A) -> (C) -> (B)\n2. (B) -> (A) -> (C)\n3....,2,NaN,"{'정답':2, 'reason': '주어진 글은 정책 평가 시 사람들이 특정 문제를...","""'policy':'정책' | 'consequence':'결과' | 'tariff'...",NaN,NaN,NaN
3,Q. 주어진 context 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오. ...,2_2303,Q. 주어진 context 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.,36,"<context>\nLike positive habits, bad habits ex...",1. (A) -> (C) -> (B)\n2. (B) -> (A) -> (C)\n3....,4,NaN,"{'정답':4, 'reason': '주어진 문맥을 바탕으로, 가장 적절한 글의 순서...","""'habit':'습관' | 'break':'깨다' | 'language':'언어'""",NaN,NaN,NaN
4,Q. 주어진 context 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.\...,2_2211,Q. 주어진 context 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.,36,<context>\nThe right to be forgotten is a righ...,1. (A) -> (C) -> (B)\n2. (B) -> (A) -> (C)\n3....,2,NaN,"{'정답':2, 'reason': '주어진 context는 ""잊혀질 권리""와 ""프라...","""'right to be forgotten':'잊혀질 권리' | 'privacy':...",NaN,NaN,NaN
5,Q. 주어진 context 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.\...,2_2312,Q. 주어진 context 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.,37,"<context>\n For a long time, random sampling w...",1. (A) -> (C) -> (B)\n2. (B) -> (A) -> (C)\n3....,3,NaN,"{'정답': 3, 'reason': '샘플링의 단점을 설명하는 (B)로 시작하여 전...","""'sampling':'샘플링' | 'data':'데이터' | 'light-fiel...",NaN,NaN,NaN
6,Q. 주어진 글 다음에 이어질 글의 순서로 가장 적절한 것을 \n고르시오.\nArc...,2_2309,Q. 주어진 글 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.,37,Architects might say a machine can never desig...,1. (A) -> (C) -> (B)\n2. (B) -> (A) -> (C)\n3....,3,NaN,"{'정답': 3, 'reason': '글의 흐름을 살펴보면, 먼저 인간만이 그렇게 ...","""'architects':'건축가들' | 'innovative':'혁신적인' | '...",NaN,NaN,NaN
7,Q. 주어진 글 다음에 이어질 글의 순서로 가장 적절한 \n것을 고르시오.\n\nS...,2_2306,Q. 주어진 글 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.,37,Species that are found in only one area are ca...,1. (A) -> (C) -> (B)\n2. (B) -> (A) -> (C)\n3....,3,NaN,"{'정답': 3, 'reason': '글의 흐름을 살펴보면, 먼저 특정 지역에만 존...","""'endemic':'고유의' | 'extinction':'멸종' | 'moistu...",NaN,NaN,NaN
8,Q. 주어진 context 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.\...,2_2303,Q. 주어진 context 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.,37,<context> \nA common but incorrect assumption ...,1. (A) -> (C) -> (B)\n2. (B) -> (A) -> (C)\n3....,3,NaN,"{'정답': 3, 'reason': '글의 흐름을 살펴보면, 먼저 우리가 이유를 나...","""'reason':'이유' | 'emotion':'감정' | 'values':'가치'""",NaN,NaN,NaN
9,Q. 주어진 context 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.\...,2_2211,Q. 주어진 context 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.,37,<context>\nTo an economist who succeeds in fig...,1. (A) -> (C) -> (B)\n2. (B) -> (A) -> (C)\n3....,3,NaN,"{'정답': 3, 'reason': '글의 흐름을 살펴보면, 먼저 경제학자가 출생률...","""'economist':'경제학자' | 'preference structure':'...",NaN,NaN,NaN


In [4]:
# 파인튜닝을 위해 jsonl 형식으로 저장하기
ft_data = []
for i in range(len(df_que_ans)):
    message = {"messages": [{"role":"system", "content": "당신은 수능영어 전문가입니다. 선생님처럼 대답해주세요."},
                            { "role": "user", "content": df_que_ans['full_question'][i]},
                            { "role": "assistant", "content": df_que_ans['solution'][i]}
                           ]
               }
    ft_data.append(message)

# 학습 데이터
# system 에는 AI의 페르소나 혹은 공통된 답변 규칙등을 설정합니다.
# user 에는 사용자의 질문이나 요청, 지시를 입력합니다.
# assistant 에는 사용자의 입력에 따른 모델의 답변을 입력합니다.

# 위에서 설정한 학습 데이터를 파일로 저장합니다.
with open(f"data/ft_data4.jsonl", "w", encoding='utf8') as json_file:
    for entry in ft_data:
        json.dump(entry, json_file, ensure_ascii=False)
        json_file.write('\n')

k = 0
with open(f"data/ft_data4.jsonl", "r", encoding='utf-8-sig') as json_file:
    for line in json_file:
        print(line)
        k = k + 1
    print("jsonl 라인 수: ", k)

{"messages": [{"role": "system", "content": "당신은 수능영어 전문가입니다. 선생님처럼 대답해주세요."}, {"role": "user", "content": "Q. 주어진 context 다음에 이어질 글의 순서로 가장 적절한 것을 고르시오.\n<context>\n The size of a species is not accidental. It’s a fine-tuned interaction between a species and the world it inhabits. Over large periods of time, size fluctuations have often signalled significant changes in the environment. </context>\n\n(A) But we are beginning to see changes in this trend. Scientists have discovered that many animals are shrinking. Around the world, species in every category have been found to be getting smaller, and one major cause appears to be the heat. \n\n(B) Generally speaking, over the last five hundred million years, the trend has been towards animals getting larger. It’s particularly notable in marine animals, whose average body size has increased 150-fold in this time. \n\n(C) Animals living in the Italian Alps, for example, have seen temperatures rise by three to four degrees Celsius since the

# 2. 파인튜닝

당연하게도 비용이 발생합니다.

파인튜닝이 끝나면 메일로 결과가 전달됩니다.

In [14]:
# 학습 데이터를 OpenAI로 전송합니다.
client_file = client.files.create(
    file=open("data/ft_data4.jsonl", "rb"),
    purpose='fine-tune'
)

# 파인튜닝 시작
ft_job = client.fine_tuning.jobs.create(
    # 아래 training_file 값을 위의 실행 결과에서 받은 id 값으로 교체
    training_file = client_file.id,
    model = "gpt-3.5-turbo",             # gpt-3.5-turbo, gpt-4-turbo, gpt-4o, fine_tuned_model_name
    # hyperparameters = {'n_epochs' : 10}
)

ft_job

# 파인튜닝 상세한 옵션은 다음 문서를 참고하세요.
# https://platform.openai.com/docs/api-reference/fine-tuning/create

FineTuningJob(id='ftjob-Hf57VTe6egoEwWyM32K8EYkx', created_at=1717219730, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-6eteSCPFQlMum3eXj8hSVwYz', result_files=[], status='validating_files', trained_tokens=None, training_file='file-FNqXg3kNrtNlukdhTpMxqfaA', validation_file=None, user_provided_suffix=None, seed=477617447, estimated_finish=None, integrations=[])

In [17]:
# 특정 파인튜닝 작업의 상태 확인
client.fine_tuning.jobs.retrieve(ft_job.id)

# 파인튜닝 중(status='running')... => 파인튜닝이 완료(succeeded)되면 구글 메일로 안내가 옴

FineTuningJob(id='ftjob-Hf57VTe6egoEwWyM32K8EYkx', created_at=1717219730, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9VBgyCEI', finished_at=1717220139, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-6eteSCPFQlMum3eXj8hSVwYz', result_files=['file-p2Bns4bNrSStsphi5Igxft5F'], status='succeeded', trained_tokens=74385, training_file='file-FNqXg3kNrtNlukdhTpMxqfaA', validation_file=None, user_provided_suffix=None, seed=477617447, estimated_finish=None, integrations=[])

In [42]:
# 파인튜닝 작업 목록 및 상태 확인
client.fine_tuning.jobs.list(limit=10)

# 특정 파인튜닝 작업의 상태 확인
# 파라미터 값은 위에서 실행한 결과의 id 값
# client.fine_tuning.jobs.retrieve("ftjob-5o3P9m7cg3AZgXYFWgn60VQv")

# 특정 파인튜닝 작업의 학습 취소
# client.fine_tuning.jobs.cancel("ftjob-5o3P9m7cg3AZgXYFWgn60VQv")

# 특정 파인튜닝 작업시 발생한 이벤트 확인
# client.fine_tuning.jobs.list_events(id="ftjob-5o3P9m7cg3AZgXYFWgn60VQv", limit=10)

# 학습한 파인튜닝 모델 삭제
# 파라미터 값은 모델 학습이 완료된 뒤, 결과로 받은 fine_tuned_model 값
# client.models.delete("ft:gpt-3.5-turbo-0613:playmore-ai::8Whhhnxu")

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-l1XPULDPxQa2kltHHzyKMRsC', created_at=1716621517, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-6eteSCPFQlMum3eXj8hSVwYz', result_files=[], status='running', trained_tokens=None, training_file='file-6PefistfZ2eEMnFWMI2n6FWi', validation_file=None, user_provided_suffix=None, seed=1580900318, estimated_finish=None, integrations=[]), FineTuningJob(id='ftjob-6P7VVUArunmY5KixblvREyo6', created_at=1716621093, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9SfvL2Ga', finished_at=1716621366, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-6eteSCPFQlMum3eXj8hSVwYz', r

# 3. 파인튜닝 모델 사용

OpenAI Playground의 모델 선택창에서 선택하여 테스트 해 볼 수 있습니다.

파인튜닝 모델의 추론 비용은 기본 모델의 약 8배 정도 입니다. 단, 시스템 메시지와 사전 사용자 메시지의 분량이 매우 많은 경우 비용면에서도 이득이 될 수 있습니다.

In [24]:
fine_tuned_model_name = client.fine_tuning.jobs.retrieve(ft_job.id).fine_tuned_model

completion = client.chat.completions.create(
  # 파인튜닝 종료 후 작업 목록 결과에서 받은 fine_tuned_model 혹은 메일로 받은 new model 값을 model 값으로 설정
  model = fine_tuned_model_name,
  messages=[
    {"role": "system", "content": "당신은 수능영어 전문가입니다. 선생님처럼 대답해주세요."},
    {"role": "user", "content": 
     """다음 글의 목적으로 가장 적절한 것은?
To whom it may concern,
I would like to draw your attention to a problem that 
frequently occurs with the No. 35 buses. There is a bus 
stop about halfway along Fenny Road, at which the No. 
35 buses are supposed to stop. It would appear, however, 
that some of your drivers are either unaware of this bus 
stop or for some reason choose to ignore it, driving past 
even though the buses are not full. I would be grateful if 
you could remind your drivers that this bus stop exists 
and that they should be prepared to stop at it. I look 
forward to seeing an improvement in this service soon. 
Yours faithfully,
John Williams
1. 버스 운전기사 채용 계획을 문의하려고
2. 버스 정류장의 위치 변경을 요청하려고
3. 도로 공사로 인한 소음에 대해 항의하려고
4. 출퇴근 시간의 버스 배차 간격 단축을 제안하려고
5. 버스 정류장 무정차 통과에 대한 시정을 요구하려고"""}
  ]
)

print(completion.choices[0].message.content)

{'정답':5, 'reason': '글의 내용을 살펴본 결과, 가장 적절한 목적은 버스 정류장 무시 문제에 대한 시정을 요구하는 것입니다. 글에서는 35번 버스 기사들이 일부 버스 정류장을 모르거나 무시하는 경향이 있다는 점을 지적하고, 이 버스 정류장이 존재하며 정차해아 한다는 점을 기사들에게 알리라고 요청합니다.'}


In [89]:
# System Message 없이 파인튜닝 모델 사용

completion = client.chat.completions.create(
  # 파인튜닝 종료 후 작업 목록 결과에서 받은 fine_tuned_model 값을 model 값으로 설정
  model = fine_tuned_model_name,
  messages = [
    {"role": "user", "content": 
     """다음 글의 목적으로 가장 적절한 것은?
To whom it may concern,
I would like to draw your attention to a problem that 
frequently occurs with the No. 35 buses. There is a bus 
stop about halfway along Fenny Road, at which the No. 
35 buses are supposed to stop. It would appear, however, 
that some of your drivers are either unaware of this bus 
stop or for some reason choose to ignore it, driving past 
even though the buses are not full. I would be grateful if 
you could remind your drivers that this bus stop exists 
and that they should be prepared to stop at it. I look 
forward to seeing an improvement in this service soon. 
Yours faithfully,
John Williams
1. 버스 운전기사 채용 계획을 문의하려고
2. 버스 정류장의 위치 변경을 요청하려고
3. 도로 공사로 인한 소음에 대해 항의하려고
4. 출퇴근 시간의 버스 배차 간격 단축을 제안하려고
5. 버스 정류장 무정차 통과에 대한 시정을 요구하려고"""}
  ]
)

print(completion.choices[0].message.content)

글의 목적은 No. 35 버스가 자주 무시하는 버스 정류장에서의 무정차 행동에 대해 주의를 끌고, 이에 대한 개선을 요구하는 것입니다. 따라서 가장 적절한 것은 ⑤번 "버스 정류장 무정차 통과에 대한 시정을 요구하려고" 입니다.



In [90]:
# 기본 모델 사용

completion = client.chat.completions.create(
  model = "gpt-3.5-turbo-0613",
  messages = [
    {"role": "user", "content": 
     """다음 글의 목적으로 가장 적절한 것은?
To whom it may concern,
I would like to draw your attention to a problem that 
frequently occurs with the No. 35 buses. There is a bus 
stop about halfway along Fenny Road, at which the No. 
35 buses are supposed to stop. It would appear, however, 
that some of your drivers are either unaware of this bus 
stop or for some reason choose to ignore it, driving past 
even though the buses are not full. I would be grateful if 
you could remind your drivers that this bus stop exists 
and that they should be prepared to stop at it. I look 
forward to seeing an improvement in this service soon. 
Yours faithfully,
John Williams
1. 버스 운전기사 채용 계획을 문의하려고
2. 버스 정류장의 위치 변경을 요청하려고
3. 도로 공사로 인한 소음에 대해 항의하려고
4. 출퇴근 시간의 버스 배차 간격 단축을 제안하려고
5. 버스 정류장 무정차 통과에 대한 시정을 요구하려고"""}
  ]
)

print(completion.choices[0].message.content)

⑤ 버스 정류장 무정차 통과에 대한 시정을 요구하려고


In [ ]:
# 파인튜닝 효과
# 1. 정답을 잘 맞춤
# 2. 선생님 같은 말투로 자세하게 설명해줌

# 4. 수능 문제 풀어보기

## 4-1. 2_2312

In [27]:
year_month = '2_2312'
df = pd.read_excel("data/data_영어/영어_모의고사_문제&답변.xlsx", sheet_name=year_month)
# df.head(3)

fine_tuned_model_name = client.fine_tuning.jobs.retrieve(ft_job.id).fine_tuned_model

# 질문을 받으면 답변하는 함수
def gpt_response(question):

    # 메시지 히스토리 생성
    messages=[{"role": "system", 
               "content":  "당신은 수능영어 전문가입니다. 선생님처럼 대답해주세요."
              }]

    question_dict = {
                "role": "user",
                "content": question + "\n 답변 양식 : {'정답':number, 'reason': 'string'}"
            }
    
    # 메시지 히스토리에 질문 추가
    messages.append(question_dict)
    
    # 답변 받기
    completion = client.chat.completions.create(
        model = fine_tuned_model_name,    # gpt-3.5-turbo, gpt-4-turbo, gpt-4-0125-preview, fine_tuned_model_name
        messages = messages
    )
    
    response = completion.choices[0].message.content
    
    return response

# test = gpt_response(df['문제'][0])
# test


# 답변을 딕셔너리로 받아서 리스트에 저장
response_list = []
print(year_month)
print(model)

print("답변 생성 시작")
for i in tqdm_notebook(range(len(df['문제']))):   # len(df['문제']))
    response = gpt_response(df['문제'][i])
#     if response[0] == "{" and response[-1] != "}":
#         response = response + "}" 
#     elif response[0] != "{" and response[-1] != "}":
#         response = str({"문제번호":18+i, "답변":0, "이유":"-"})
#     print(response, '\n')
    response_list.append(response)
print("답변 생성 종료")

# print(len(response_list), "문제")
# response_list

2_2312
None
답변 생성 시작


  0%|          | 0/28 [00:00<?, ?it/s]

답변 생성 종료


In [29]:
# 답변 번호, 이유 추출하기
answer_list = []
reason_list = []

for j in range(len(response_list)):
#     print(18+j, "번 문제")
    
    # 답변 이유 추출하기
    try:      
        reason = response_list[j].split('reason')[1].replace(":", "").replace("\'", "").replace("}", "")    
    except:
        reason = "문제가 명확하지 않습니다. 정답 번호는 0"
    reason_list.append(reason)
#     print(reason, '\n')
    
    # 답변 번호 추출하다가 에러나면 다시 gpt에게 답변 번호 물어보기    
    try:
        raw_number = response_list[j].split('reason')[0]
    #     print(raw_number)

        if '①' in raw_number:
            number = 1
        elif '②' in raw_number:
            number = 2
        elif '③' in raw_number:
            number = 3
        elif '④' in raw_number:
            number = 4
        elif '⑤' in raw_number:
            number = 5
        else:
            number = int(re.sub(r'[^0-9]', '', raw_number))
    except:
    #     print(j, "에러 발생", '\n')
        re_response = gpt_response(reason + " -> 정답 번호만 숫자로 알려줘. 답변 양식 : int(number)")
        number = int(re.sub(r'[^0-9]', '', re_response.split('reason')[0]))
        
#     print("정답:", number, '\n')    
#     print("---------------------------------------------", '\n')
    answer_list.append(number)
    
#     print(j)
#     raw_number = response_list[j].split('reason')[0]
#     number = int(re.sub(r'[^0-9]', '', raw_number))
# #     print(number, '\n')
#     answer_list.append(number)

#     reason = response_list[j].split('reason')[1].replace(":", "").replace("\'", "").replace("}", "")
# #     print(reason)
#     reason_list.append(reason)

# 답변과 정답 맞춰보기
right = df['정답'] == answer_list

# print(len(reason_list))
# reason_list

df_final = pd.DataFrame({"questionNumber":range(18, 46), "CorrectAnswer":df['정답'], "answer":answer_list, "right":right, "TypeString":df['TypeString'], "reason":reason_list})

print(year_month)
# 맞춘 갯수
print("맞춘 개수:", sum(df_final['right'].values))

# 틀린 갯수
print("틀린 개수:", 28 - sum(df_final['right'].values))

# 틀린 문제 유형
print("틀린 문제 유형: ")
globals()[f'df_final_{year_month}'] = df_final[df_final['right'] == False][['questionNumber', 'TypeString']]
globals()[f'df_final_{year_month}']

2_2312
맞춘 개수: 23
틀린 개수: 5
틀린 문제 유형: 


,questionNumber,TypeString
7,25,그래프 해석
17,35,글의 흐름과 무관한 문장 찾기
18,36,글의 순서 파악
24,42,문맥상 적절하지 않은 어휘 고르기
26,44,지칭 대상 파악


# 4-2. 2_2309

In [39]:
%pwd

'C:\\Users\\runia\\Desktop\\[공모전] gpt_수능영어_파인튜닝'

In [42]:
year_month = '2_2309'
df = pd.read_excel("data/data_영어/영어_모의고사_문제&답변.xlsx", sheet_name=year_month)
# df.head(3)

fine_tuned_model_name = client.fine_tuning.jobs.retrieve(ft_job.id).fine_tuned_model

# 질문을 받으면 답변하는 함수
def gpt_response(question):

    # 메시지 히스토리 생성
    messages=[{"role": "system", 
               "content":  "당신은 수능영어 전문가입니다. 선생님처럼 대답해주세요."
              }]

    question_dict = {
                "role": "user",
                "content": question + "\n 답변 양식 : {'정답':number, 'reason': 'string'}"
            }
    
    # 메시지 히스토리에 질문 추가
    messages.append(question_dict)
    
    # 답변 받기
    completion = client.chat.completions.create(
        model = fine_tuned_model_name,    # gpt-3.5-turbo, gpt-4-turbo, gpt-4-0125-preview, fine_tuned_model_name
        messages = messages
    )
    
    response = completion.choices[0].message.content
    
    return response

# test = gpt_response(df['문제'][0])
# test


# 답변을 딕셔너리로 받아서 리스트에 저장
response_list = []
print(year_month)
print(model)

print("답변 생성 시작")
for i in tqdm_notebook(range(len(df['문제']))):   # len(df['문제']))
    response = gpt_response(df['문제'][i])
#     if response[0] == "{" and response[-1] != "}":
#         response = response + "}" 
#     elif response[0] != "{" and response[-1] != "}":
#         response = str({"문제번호":18+i, "답변":0, "이유":"-"})
#     print(response, '\n')
    response_list.append(response)
print("답변 생성 종료")

# print(len(response_list), "문제")
# response_list

2_2309
None
답변 생성 시작


  0%|          | 0/28 [00:00<?, ?it/s]

답변 생성 종료


In [45]:
# 답변 번호, 이유 추출하기
answer_list = []
reason_list = []

for j in range(len(response_list)):
#     print(18+j, "번 문제")
    
    # 답변 이유 추출하기
    try:      
        reason = response_list[j].split('reason')[1].replace(":", "").replace("\'", "").replace("}", "")    
    except:
        reason = "문제가 명확하지 않습니다. 정답 번호는 0"
    reason_list.append(reason)
#     print(reason, '\n')
    
    # 답변 번호 추출하다가 에러나면 다시 gpt에게 답변 번호 물어보기    
    try:
        raw_number = response_list[j].split('reason')[0]
    #     print(raw_number)

        if '①' in raw_number:
            number = 1
        elif '②' in raw_number:
            number = 2
        elif '③' in raw_number:
            number = 3
        elif '④' in raw_number:
            number = 4
        elif '⑤' in raw_number:
            number = 5
        else:
            number = int(re.sub(r'[^0-9]', '', raw_number))
    except:
    #     print(j, "에러 발생", '\n')
        re_response = gpt_response(reason + " -> 정답 번호만 숫자로 알려줘. 답변 양식 : int(number)")
        number = int(re.sub(r'[^0-9]', '', re_response.split('reason')[0]))
        
#     print("정답:", number, '\n')    
#     print("---------------------------------------------", '\n')
    answer_list.append(number)
    
#     print(j)
#     raw_number = response_list[j].split('reason')[0]
#     number = int(re.sub(r'[^0-9]', '', raw_number))
# #     print(number, '\n')
#     answer_list.append(number)

#     reason = response_list[j].split('reason')[1].replace(":", "").replace("\'", "").replace("}", "")
# #     print(reason)
#     reason_list.append(reason)

# 답변과 정답 맞춰보기
right = df['정답'] == answer_list

# print(len(reason_list))
# reason_list

df_final = pd.DataFrame({"questionNumber":range(18, 46), "CorrectAnswer":df['정답'], "answer":answer_list, "right":right, "TypeString":df['TypeString'], "reason":reason_list})

print(year_month)
# 맞춘 갯수
print("맞춘 개수:", sum(df_final['right'].values))

# 틀린 갯수
print("틀린 개수:", 28 - sum(df_final['right'].values))

# 틀린 문제 유형
print("틀린 문제 유형: ")
globals()[f'df_final_{year_month}'] = df_final[df_final['right'] == False][['questionNumber', 'TypeString']]
globals()[f'df_final_{year_month}']

2_2309
맞춘 개수: 21
틀린 개수: 7
틀린 문제 유형: 


,questionNumber,TypeString
3,21,함축 의미 추론
7,25,그래프 해석
12,30,문맥상 적절하지 않은 어휘 고르기
13,31,빈칸 내용 추론
17,35,글의 흐름과 무관한 문장 찾기
24,42,문맥상 적절하지 않은 어휘 고르기
26,44,지칭 대상 파악


# 4-3. 2_2209

In [46]:
year_month = '2_2209'
df = pd.read_excel("data/data_영어/영어_모의고사_문제&답변.xlsx", sheet_name=year_month)
# df.head(3)

fine_tuned_model_name = client.fine_tuning.jobs.retrieve(ft_job.id).fine_tuned_model

# 질문을 받으면 답변하는 함수
def gpt_response(question):

    # 메시지 히스토리 생성
    messages=[{"role": "system", 
               "content":  "당신은 수능영어 전문가입니다. 선생님처럼 대답해주세요."
              }]

    question_dict = {
                "role": "user",
                "content": question + "\n 답변 양식 : {'정답':number, 'reason': 'string'}"
            }
    
    # 메시지 히스토리에 질문 추가
    messages.append(question_dict)
    
    # 답변 받기
    completion = client.chat.completions.create(
        model = fine_tuned_model_name,    # gpt-3.5-turbo, gpt-4-turbo, gpt-4-0125-preview, fine_tuned_model_name
        messages = messages
    )
    
    response = completion.choices[0].message.content
    
    return response

# test = gpt_response(df['문제'][0])
# test


# 답변을 딕셔너리로 받아서 리스트에 저장
response_list = []
print(year_month)
print(model)

print("답변 생성 시작")
for i in tqdm_notebook(range(len(df['문제']))):   # len(df['문제']))
    response = gpt_response(df['문제'][i])
#     if response[0] == "{" and response[-1] != "}":
#         response = response + "}" 
#     elif response[0] != "{" and response[-1] != "}":
#         response = str({"문제번호":18+i, "답변":0, "이유":"-"})
#     print(response, '\n')
    response_list.append(response)
print("답변 생성 종료")

# print(len(response_list), "문제")
# response_list

2_2209
None
답변 생성 시작


  0%|          | 0/28 [00:00<?, ?it/s]

답변 생성 종료


In [47]:
# 답변 번호, 이유 추출하기
answer_list = []
reason_list = []

for j in range(len(response_list)):
#     print(18+j, "번 문제")
    
    # 답변 이유 추출하기
    try:      
        reason = response_list[j].split('reason')[1].replace(":", "").replace("\'", "").replace("}", "")    
    except:
        reason = "문제가 명확하지 않습니다. 정답 번호는 0"
    reason_list.append(reason)
#     print(reason, '\n')
    
    # 답변 번호 추출하다가 에러나면 다시 gpt에게 답변 번호 물어보기    
    try:
        raw_number = response_list[j].split('reason')[0]
    #     print(raw_number)

        if '①' in raw_number:
            number = 1
        elif '②' in raw_number:
            number = 2
        elif '③' in raw_number:
            number = 3
        elif '④' in raw_number:
            number = 4
        elif '⑤' in raw_number:
            number = 5
        else:
            number = int(re.sub(r'[^0-9]', '', raw_number))
    except:
    #     print(j, "에러 발생", '\n')
        re_response = gpt_response(reason + " -> 정답 번호만 숫자로 알려줘. 답변 양식 : int(number)")
        number = int(re.sub(r'[^0-9]', '', re_response.split('reason')[0]))
        
#     print("정답:", number, '\n')    
#     print("---------------------------------------------", '\n')
    answer_list.append(number)
    
#     print(j)
#     raw_number = response_list[j].split('reason')[0]
#     number = int(re.sub(r'[^0-9]', '', raw_number))
# #     print(number, '\n')
#     answer_list.append(number)

#     reason = response_list[j].split('reason')[1].replace(":", "").replace("\'", "").replace("}", "")
# #     print(reason)
#     reason_list.append(reason)

# 답변과 정답 맞춰보기
right = df['정답'] == answer_list

# print(len(reason_list))
# reason_list

df_final = pd.DataFrame({"questionNumber":range(18, 46), "CorrectAnswer":df['정답'], "answer":answer_list, "right":right, "TypeString":df_final['TypeString'], "reason":reason_list})

print(year_month)
# 맞춘 갯수
print("맞춘 개수:", sum(df_final['right'].values))

# 틀린 갯수
print("틀린 개수:", 28 - sum(df_final['right'].values))

# 틀린 문제 유형
print("틀린 문제 유형: ")
globals()[f'df_final_{year_month}'] = df_final[df_final['right'] == False][['questionNumber', 'TypeString']]
globals()[f'df_final_{year_month}']

2_2209
맞춘 개수: 15
틀린 개수: 13
틀린 문제 유형: 


,questionNumber,TypeString
7,25,그래프 해석
9,27,글의 세부 내용 파악
14,32,빈칸 내용 추론
17,35,글의 흐름과 무관한 문장 찾기
18,36,글의 순서 파악
19,37,글의 순서 파악
20,38,문장 삽입
21,39,문장 삽입
22,40,글의 내용 요약
24,42,문맥상 적절하지 않은 어휘 고르기


# 4-4. 2_2203

In [376]:
year_month = '2_2203'
df = pd.read_excel("data/data_영어/영어_모의고사_문제&답변.xlsx", sheet_name=year_month)
# df.head(3)

fine_tuned_model_name = client.fine_tuning.jobs.retrieve(ft_job.id).fine_tuned_model

# 질문을 받으면 답변하는 함수
def gpt_response(question):

    # 메시지 히스토리 생성
    messages=[{"role": "system", 
               "content":  "당신은 수능영어 전문가입니다. 선생님처럼 대답해주세요."
              }]

    question_dict = {
                "role": "user",
                "content": question + "\n 답변 양식 : {'정답':number, 'reason': 'string'}"
            }
    
    # 메시지 히스토리에 질문 추가
    messages.append(question_dict)
    
    # 답변 받기
    completion = client.chat.completions.create(
        model = fine_tuned_model_name,    # gpt-3.5-turbo, gpt-4-turbo, gpt-4-0125-preview, fine_tuned_model_name
        messages = messages
    )
    
    response = completion.choices[0].message.content
    
    return response

# test = gpt_response(df['문제'][0])
# test


# 답변을 딕셔너리로 받아서 리스트에 저장
response_list = []
print(year_month)
print(model)

print("답변 생성 시작")
for i in tqdm_notebook(range(len(df['문제']))):   # len(df['문제']))
    response = gpt_response(df['문제'][i])
#     if response[0] == "{" and response[-1] != "}":
#         response = response + "}" 
#     elif response[0] != "{" and response[-1] != "}":
#         response = str({"문제번호":18+i, "답변":0, "이유":"-"})
#     print(response, '\n')
    response_list.append(response)
print("답변 생성 종료")

# print(len(response_list), "문제")
# response_list

2_2203
ft:gpt-3.5-turbo-0125:personal::9THs20d9
답변 생성 시작


  0%|          | 0/28 [00:00<?, ?it/s]

답변 생성 종료


In [380]:
# 답변 번호, 이유 추출하기
answer_list = []
reason_list = []

for j in range(len(response_list)):
#     print(18+j, "번 문제")
    
    # 답변 이유 추출하기
    try:      
        reason = response_list[j].split('reason')[1].replace(":", "").replace("\'", "").replace("}", "")    
    except:
        reason = "문제가 명확하지 않습니다. 정답 번호는 0"
    reason_list.append(reason)
#     print(reason, '\n')
    
    # 답변 번호 추출하다가 에러나면 다시 gpt에게 답변 번호 물어보기    
    try:
        raw_number = response_list[j].split('reason')[0]
    #     print(raw_number)

        if '①' in raw_number:
            number = 1
        elif '②' in raw_number:
            number = 2
        elif '③' in raw_number:
            number = 3
        elif '④' in raw_number:
            number = 4
        elif '⑤' in raw_number:
            number = 5
        else:
            number = int(re.sub(r'[^0-9]', '', raw_number))
    except:
    #     print(j, "에러 발생", '\n')
        re_response = gpt_response(reason + " -> 정답 번호만 숫자로 알려줘. 답변 양식 : int(number)")
        number = int(re.sub(r'[^0-9]', '', re_response.split('reason')[0]))
        
#     print("정답:", number, '\n')    
#     print("---------------------------------------------", '\n')
    answer_list.append(number)
    
#     print(j)
#     raw_number = response_list[j].split('reason')[0]
#     number = int(re.sub(r'[^0-9]', '', raw_number))
# #     print(number, '\n')
#     answer_list.append(number)

#     reason = response_list[j].split('reason')[1].replace(":", "").replace("\'", "").replace("}", "")
# #     print(reason)
#     reason_list.append(reason)

# 답변과 정답 맞춰보기
right = df['정답'] == answer_list

# print(len(reason_list))
# reason_list

df_final = pd.DataFrame({"questionNumber":range(18, 46), "CorrectAnswer":df['정답'], "answer":answer_list, "right":right, "TypeString":df_final['TypeString'], "reason":reason_list})

print(year_month)
# 맞춘 갯수
print("맞춘 개수:", sum(df_final['right'].values))

# 틀린 갯수
print("틀린 개수:", 28 - sum(df_final['right'].values))

# 틀린 문제 유형
print("틀린 문제 유형: ")
globals()[f'df_final_{year_month}'] = df_final[df_final['right'] == False][['questionNumber', 'TypeString']]
globals()[f'df_final_{year_month}']

18 번 문제
 이 편지의 주된 목적은 "다른 날짜로 기차표 변경을 요청하려고" 합니다. 글에서 주장하는 것은 특가 티켓이 원래 환불이 불가능하다는 점을 이해하고 있지만, 축구 경기로 인해 여행 일정을 바꿔야 한다는 점입니다. 따라서 이 편지는 단순히 특가로 기차표를 구매하려는 게 아니라, 이미 구매한 특가 티켓의 일정 변경을 요구하는 내용을 담고 있습니다. 

정답: 3 

--------------------------------------------- 

19 번 문제
 "이 문제에서 I는 이벤트(패션쇼) 직전 자신의 심리상태를 묘사하고 있습니다. 문장을 통해 I의 손바닥에 땀이 나고 복부에는 나비가 날고 있다고 설명되었습니다. 또한 모델들이 이미 19세기 의상을 입은 상태라는 점도 I가 시간이 늦었다는 압박을 느끼게 만들었습니다.

이를 바탕으로 I의 심경으로 가장 적절한 것은

1번 tense and nervous - 긴장되어 있고 불안한. 이는 문장에서 I가 패션쇼를 앞둔 긴장감과 불안을 느끼고 있는 상태를 묘사했기 때문입니다.

따라서, 정답은 1번 tense and nervous입니다." 

정답: 1 

--------------------------------------------- 

20 번 문제
 글에서 필자가 주장하는 바를 가장 잘 나타내는 선택은 3입니다. 필자는 다른 문화에 노출되는 경험이 자녀들의 창의성을 향상시키고 개방적인 사고를 육성하는 데 중요하다고 강조합니다. 부모님들은 자녀들을 다른 문화와 자주 접촉하도록 유도해아며, 가능하다면 다른 나라로 여행을 나가거나 거기서 생활하라고 조언합니다. 이는 선택지 3의 내용과 일치하며, 필자의 주장을 정확하게 반영하고 있습니다. 

정답: 3 

--------------------------------------------- 

21 번 문제
 문제에서 <Fish is Fish­style assimilation>은 어린이들이 새로운 정보(지구가 둥글다는 사실)를 기존의 사고 방식(지구

,questionNumber,TypeString
7,25,그래프 해석
10,28,글의 세부 내용 파악
17,35,글의 흐름과 무관한 문장 찾기
19,37,글의 순서 파악
20,38,문장 삽입
24,42,문맥상 적절하지 않은 어휘 고르기
26,44,지칭 대상 파악


# 4-5. 2_2111

In [382]:
year_month = '2_2111'
df = pd.read_excel("data/data_영어/영어_모의고사_문제&답변.xlsx", sheet_name=year_month)
# df.head(3)

fine_tuned_model_name = client.fine_tuning.jobs.retrieve(ft_job.id).fine_tuned_model

# 질문을 받으면 답변하는 함수
def gpt_response(question):

    # 메시지 히스토리 생성
    messages=[{"role": "system", 
               "content":  "당신은 수능영어 전문가입니다. 선생님처럼 대답해주세요."
              }]

    question_dict = {
                "role": "user",
                "content": question + "\n 답변 양식 : {'정답':number, 'reason': 'string'}"
            }
    
    # 메시지 히스토리에 질문 추가
    messages.append(question_dict)
    
    # 답변 받기
    completion = client.chat.completions.create(
        model = fine_tuned_model_name,    # gpt-3.5-turbo, gpt-4-turbo, gpt-4-0125-preview, fine_tuned_model_name
        messages = messages
    )
    
    response = completion.choices[0].message.content
    
    return response

# test = gpt_response(df['문제'][0])
# test


# 답변을 딕셔너리로 받아서 리스트에 저장
response_list = []
print(year_month)
print(model)

print("답변 생성 시작")
for i in tqdm_notebook(range(len(df['문제']))):   # len(df['문제']))
    response = gpt_response(df['문제'][i])
#     if response[0] == "{" and response[-1] != "}":
#         response = response + "}" 
#     elif response[0] != "{" and response[-1] != "}":
#         response = str({"문제번호":18+i, "답변":0, "이유":"-"})
#     print(response, '\n')
    response_list.append(response)
print("답변 생성 종료")

# print(len(response_list), "문제")
# response_list

2_2111
ft:gpt-3.5-turbo-0125:personal::9THs20d9
답변 생성 시작


  0%|          | 0/28 [00:00<?, ?it/s]

답변 생성 종료


In [406]:
# 답변 번호, 이유 추출하기
answer_list = []
reason_list = []

for j in range(len(response_list)):
#     print(18+j, "번 문제")
    
    # 답변 이유 추출하기
    try:      
        reason = response_list[j].split('reason')[1].replace(":", "").replace("\'", "").replace("}", "")    
    except:
        reason = "문제가 명확하지 않습니다. 정답 번호는 0"
    reason_list.append(reason)
#     print(reason, '\n')
    
    # 답변 번호 추출하다가 에러나면 다시 gpt에게 답변 번호 물어보기    
    try:
        raw_number = response_list[j].split('reason')[0]
    #     print(raw_number)

        if '①' in raw_number:
            number = 1
        elif '②' in raw_number:
            number = 2
        elif '③' in raw_number:
            number = 3
        elif '④' in raw_number:
            number = 4
        elif '⑤' in raw_number:
            number = 5
        else:
            number = int(re.sub(r'[^0-9]', '', raw_number))
    except:
    #     print(j, "에러 발생", '\n')
        re_response = gpt_response(reason + " -> 정답 번호만 숫자로 알려줘. 답변 양식 : int(number)")
        number = int(re.sub(r'[^0-9]', '', re_response.split('reason')[0]))
        
#     print("정답:", number, '\n')    
#     print("---------------------------------------------", '\n')
    answer_list.append(number)
    
#     print(j)
#     raw_number = response_list[j].split('reason')[0]
#     number = int(re.sub(r'[^0-9]', '', raw_number))
# #     print(number, '\n')
#     answer_list.append(number)

#     reason = response_list[j].split('reason')[1].replace(":", "").replace("\'", "").replace("}", "")
# #     print(reason)
#     reason_list.append(reason)

# 답변과 정답 맞춰보기
right = df['정답'] == answer_list

# print(len(reason_list))
# reason_list

df_final = pd.DataFrame({"questionNumber":range(18, 46), "CorrectAnswer":df['정답'], "answer":answer_list, "right":right, "TypeString":df_final['TypeString'], "reason":reason_list})

print(year_month)
# 맞춘 갯수
print("맞춘 개수:", sum(df_final['right'].values))

# 틀린 갯수
print("틀린 개수:", 28 - sum(df_final['right'].values))

# 틀린 문제 유형
print("틀린 문제 유형: ")
globals()[f'df_final_{year_month}'] = df_final[df_final['right'] == False][['questionNumber', 'TypeString']]
globals()[f'df_final_{year_month}']

2_2111
맞춘 개수: 17
틀린 개수: 11
틀린 문제 유형


,questionNumber,TypeString
7,25,그래프 해석
8,26,글의 세부 내용 파악
11,29,어법상 틀린 표현 고르기
18,36,글의 순서 파악
19,37,글의 순서 파악
20,38,문장 삽입
21,39,문장 삽입
24,42,문맥상 적절하지 않은 어휘 고르기
25,43,장문 글순서 파악
26,44,지칭 대상 파악
